# **Virtual Pet Game Timmy Tinker, Main File Code and Documentation**

In the following code, we have more detailed descriptions and markdown comments on our main project file, which together with the in-code comments are supposed to serve as a documentation for our project. Further notes on our goal, our challenges, possible improvements and our reflections can be found in a seperate pdf file within the repository.
<br>All our project files can also be accessed online via our GitHub repository: https://github.com/TamagochiSS/Tamagotchi 

The following block includes all of our imports, which are the libraries we used for our project. Furthermore, a constant is defined to be able to save the pet data later on, if the user wishes to do so and for the customtkinter, appearance and color theme are set.

In [7]:
import tkinter as tk
from tkinter import messagebox, simpledialog 
from PIL import Image, ImageTk  # ImageTk for displaying images
import time 
import json  # Lib for the JSON file format
import os  # Library to help with the save utility
import random 
import customtkinter as ctk #extension to tkinter, making it more pretty
import subprocess  # starts new process to open other file

SAVE_FILE = "pets_data.json"

ctk.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
ctk.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue")

Now we have the the first one of two classes we used to implement our project. The class **VirtualPet** is used to initialize a pet object. Thus it contains all methods having an impact on the pets status such as feeding, playing, etc. An overview of all functions in this class can be found in the following table:<br>
<table>
    <thead> 
        <tr>
            <th>method</th>
            <th>description</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>def __init__(self, name, age=0, hunger=50, happiness=50, health=50, tiredness=20, intellect=0)</td>
            <td>In the constructor, we're using a bunch of set default parameters, which can be predefined like that, as they will be the same for all objects of this class. The name however is typed in by the user, so it has to be passed as an argument when initializing the pet object.</td>
        </tr>
        <tr>
            <td>def feed(self, food_type)</td>
            <td>Called when pet is fed, it will change the status of the pet depending on the food_type. Pizza e.g. will decrease health, while salad increases health. </td>
        </tr>
        <tr>
            <td>def play(self, game_type)</td>
            <td>Called when pet is played with, will change the status of the pet depending on the game_type, e.g. "hide and seek" has the largest influence on happiness whereas "memory" also increases the intellect value of the pet. Depending on the game_type one of the three functions below is called.</td>
        </tr>
        <tr>
            <td>def memory_game(self)<br>def hide_and_seek(self)<br>def ball_game(self)</td>
            <td>Each of these methods will open a different actual mini game (3 in total). All games are implemented in several files and are further explained in their documentation files.</td>
        </tr>
        <tr>
            <td>def random_event(self)</td>
            <td>Every play action will also cause a random event, which is not linked to the game_type, but will lead to even more change in the pets data.</td>
        </tr>
        <tr>
            <td>def TV(self)</td>
            <td>Called when pet is watching TV. The pet can't decide on a TV show, instead a random show type is choosen leading to a change of the pet's status depending on the title. All possible shows are stored in a dictionary.</td>
        </tr>
        <tr>
            <td>def vet(self)</td>
            <td>Called when the pet is sent to the vet, increases health, but decreases happiness.</td>
        </tr>
        <tr>
            <td>def sleep(self)</td>
            <td>Called when the pet is sent to sleep, keeps tiredness low (which is good), increases health and decreases happiness.</td>
        </tr>
        <tr>
            <td>def show_status(self)</td>
            <td>Method to return the current status of the pet as a string, which later is given out in the label.</td>
        </tr>
        <tr>
            <td>def to_dict(self)</td>
            <td>Method which returns all pet data belonging to one pet object as a dictionary as well as real and in game time (aka pet time)</td>
        </tr>
        <tr>
            <td>def from_dict(data)</td>
            <td>A static method as it is not bound to one specific object. Receives a dictionairy with data (such as the one returned by the function above and returns a VirtualPet object inititalized with all the given data.</td>
        </tr>
</table>

<br>

In [8]:
class VirtualPet:
    '''
    VirtualPet class, holding all functions to change the pets properties/statistics
    '''

    def __init__(self, name, age=0, hunger=50, happiness=50, health=50, tiredness=20, intellect=0):
        ''' Constructor to initialize a VirtualPet instance, used with default parameters as all pets are supposed
        to start with the same preconditions.
        '''
        self.name = name
        self.age = age  # In pet days
        self.hunger = hunger
        self.happiness = happiness
        self.health = health
        self.tiredness = tiredness
        self.intellect = intellect


    def feed(self, food_type):  
        '''
        feed VirtualPet and change scores according to food choice
        '''
        if food_type == "pizza":
            self.hunger = max(0, self.hunger - 20)
            self.health = max(0, self.health - 10)
            self.happiness = min(100, self.happiness + 5)
            self.tiredness = min(100, self.tiredness + 5)
        elif food_type == "salad":
            self.hunger = max(0, self.hunger - 15)
            self.health = min(100, self.health + 5)
            self.tiredness = max(0, self.tiredness - 5)
        elif food_type == "barbecue":
            self.hunger = max(0, self.hunger - 15)
            self.happiness = min(100, self.happiness + 10)
            self.tiredness = min(100, self.tiredness + 5)

        return f"You ate {food_type} with {self.name}."

    def play(self, game_type):  
        '''
        play with VirtualPet. pet statistics change depending on game_type
        '''
        if game_type == "hide and seek":
            self.happiness = min(100, self.happiness + 20)  # score does not exceed a value of 100
            self.hunger = min(100, self.hunger + 10)
            self.tiredness = min(100, self.tiredness + 5)
            self.hide_and_seek_game()
        elif game_type == "memory":
            self.happiness = min(100, self.happiness + 15)
            self.hunger = min(100, self.hunger + 5)
            self.tiredness = min(100, self.tiredness + 5)
            self.intellect = min(100, self.intellect + 10)
            self.memory_game()
        elif game_type == "beachball":
            self.happiness = min(100, self.happiness + 10)
            self.health = min(100, self.health + 5)
            self.hunger = min(100, self.hunger + 5)
            self.tiredness = min(100, self.tiredness + 10)
            self.ball_game()

        # Call the random_event function after playing to print random event to the label/console of the game
        event_played = self.random_event()

        return f"You played {game_type} with {self.name}. {event_played}"

    
    def memory_game(self):
        '''
        starting memory game in a separate process (memory_game.py)
        '''
        subprocess.Popen(["python", "memory_game.py"])

    
    def hide_and_seek_game(self):
        '''
        starting hide and seek game in a separate process (hide_and_seek.py)
        '''
        subprocess.Popen(["python", "hide_and_seek.py"])

    
    def ball_game(self):
        '''
        starting hide and seek game in a separate process (ball_game.py)
        '''
        subprocess.Popen(["python", "ball_game.py"])


    def random_event(self):  
        '''
        when VirtualPet plays, a random event is caused, which changes both the scores and prints the event to the console
        '''
        events = ["saw a seldom bird", "stumbled and got hurt", "nothing happened"]
        event = random.choice(events) #choose random event
        if event == "saw a seldom bird":
            self.happiness = min(100, self.happiness + 10)
            self.intellect = min(100, self.intellect + 5)
            return f"{self.name} saw a seldom bird and feels super happy!"
        elif event == "stumbled and got hurt":
            self.health = max(0, self.health - 10)  # score does not fall below a value of 0
            self.intellect = max(0, self.intellect - 10)
            return f"{self.name} lost balance and fell during playing. You should get a plaster and some sweets."
        elif event == "nothing happened":
            return f"{self.name} is very calm today."

    def TV(self): 
        '''
        when VirtualPet watches TV, a random show gets picked from the dictonary and the scores are adjusted accordingly. The show is printed in the console
        '''
        tv_show = [
            {"title": "cartoon", "happiness": 10, "hunger": 10, "tiredness": 5, "intellect": -10, "health": -5,
             "message": "It is so funny and entertaining!"},
            {"title": "documentary", "happiness": 10, "hunger": 5, "tiredness": 5, "intellect": 10, "health": -5,
             "message": "Exciting facts on nature are revealed!"},
            {"title": "a home workout channel", "happiness": 10, "hunger": 5, "tiredness": 15, "intellect": 5,
             "health": 10, "message": "Quite hard to keep up with the pace!"}
        ]

        show = random.choice(tv_show)
        self.happiness = min(100, self.happiness + show["happiness"])
        self.hunger = min(100, self.hunger + show["hunger"])
        self.tiredness = min(100, self.tiredness + show["tiredness"])
        self.intellect = min(100, max(0, self.intellect + show[
            "intellect"]))  # making sure that the score does not exceed 100 and does not fall below 0
        self.health = min(100, max(0, self.health + show["health"]))
        return f"{self.name} is watching {show['title']}. {show['message']}"
    

    def vet(self):
        '''
        send VirtualPet to the vet, improve health, decreases happiness points
        '''
        self.health = min(100, self.health + 20)
        self.happiness = max(0, self.happiness - 10)
        return f"{self.name} visited the vet."
    

    def sleep(self):
        '''
        send VirtualPet to sleep. Reduces happiness and tiredness, but improves health.
        '''
        self.tiredness = max(0, self.tiredness - 40)
        self.health = min(100, self.health + 10)
        self.happiness = max(0, self.happiness - 10)
        return f"{self.name} had a good sleep."
    

    def show_status(self):
        '''
        retrieve the current status of your pet
        '''
        return (f"{self.name}'s status:\n"
                f"  Age: {self.age} days\n"
                f"  Hunger: {self.hunger}\n"
                f"  Happiness: {self.happiness}\n"
                f"  Health: {self.health}\n"
                f"  Tiredness: {self.tiredness}\n"
                f"  Intellect: {self.intellect}")
    

    def to_dict(self, real_time_elapsed, pet_time_elapsed):
        '''
        returns a dictionary containing all pet data.
        '''
        return {
            'name': self.name,
            'age': self.age,
            'hunger': self.hunger,
            'happiness': self.happiness,
            'tiredness': self.tiredness,
            'health': self.health,
            'intellect': self.intellect,
            'real_time_elapsed': real_time_elapsed,
            'pet_time_elapsed': pet_time_elapsed,
            'selected_animal': self.selected_animal  # Save the selected animal
        }

    @staticmethod
    def from_dict(data):
        pet = VirtualPet(data['name'], data['age'], data['hunger'], data['happiness'], data['tiredness'],
                         data['health'], data['intellect'])
        pet.selected_animal = data.get('selected_animal', 'cat')  #  Default to 'cat' if not found
        return pet



The class **VirtualPetApp()** implements the graphical user interface and holds most of the game logic, meaning it defines, which actions follow on which user inputs (button clicks).


method | description 
---------|----------
def \__init__(self, root)| Constructor to initialize the game. Root is a toplevel Tkinter widget, which is used for the initial set up of the Virtual Pet App.
def load_images(self)| method to load the pet images used in the beginning of the game to let the user choose a pet as well as the images used on the buttons, scales images as well.
def create_widgets(self)| Creates all buttons and widgets used for the game and divides the graphical user interface into several components such as top, middle, bottom and time frame. Within these frames, the buttons, labels, etc. are placed.
def show_play_buttons(self)| Method which 'packs' all play buttons (Play Hide and Seek, Play Memory, Play Beachball, Stop Playing) on the screen, so the user can choose what kind of game the pet should play or if it should stop playing.
def remove_play_buttons(self)| Method to remove all the play buttons from the screen, when the pet is busy with other activities.
def show_feed_buttons(self)| Method which 'packs' all feed buttons (Feed Pizza, Feed Salad, Feed Barbecue, Stop Eating) on the screen, so the user can choose what kind of food the pet should get or if it should stop eating.
def remove_feed_buttons(self)| Method to remove all the feed buttons from the screen, when the pet is busy with other activities.
def start(self)| Method to enable all the previous defined buttons as soon as start is clicked. Changes button state from disabled to normal for all buttons.
def pet_not_found(self)| Method to handle a scenario in which the pet is not found.
def create_animal_selection_widgets(self)| Method to create widgets that allow the user to choose a pet by clicking on the image button with favourite pet.
def select_animal(self, animal)| Method to set selected_animal attribute and to remove unused buttons from screen.
def create_name_widgets(self)|Method which is called, when animal is choosen. Packs start button on screen and therefore allows the user to start playing.
def load_pet_prompt(self)| If the user already played before, this method allows him/her to load the pet thay previously played with by typing in the name of the pet.
def load_pet(self, pet_name)| Method used to load a pet that was previously saved into the game by using the pet_name.
def save_pet(self)| Method used to save the current pet's data as a JSON file in order to allow the user to continue playing at a later time.
def update_times(self)| Method that stes how the actual time passes.
def update_pet_time(self)| Method that moderates the pet time (artificial in-game time). Calls birthday method every 10 pet days.
def format_real_time(self, seconds)| Function to set the format for the actual time passed.
def format_pet_time(self, pet_seconds)| Formats the pet time (artificial in-game time).
def feed(self, food_type)| Method called by clicking the "feed x" button. Calls the feed method of the VirtualPet class to enable the current pet object to undergo the status changes triggered by the chosen action. Also advances in-game time by 2 hours.
def play(self, game_type)|  Method called by clicking the "play x" button. Calls the play method of the VirtualPet class to enable the current pet object to undergo the status changes triggered by the chosen action. Also advances in-game time by 2 hours.
def TV(self)| Method called by clicking the TV button. Calls the TV method of the VirtualPet class to enable the current pet object to undergo the status changes triggered by the chosen action. Also advances in-game time by 2 hours.
def sleep(self)| Method called by clicking the sleep button. Calls the sleep method of the VirtualPet class to enable the current pet object to undergo the status changes triggered by the chosen action. Also advances in-game time by 2 hours.
def vet(self)| Method called by clicking the "vet" button. Calls the vet method of the VirtualPet class to enable the current pet object to undergo the status changes triggered by the chosen action. Also advances in-game time by 2 hours.
def advance_pet_time(self, seconds)| Method to provide an output to the console on each new pet day.
def celebrate_birthday(self)| Method to handle the pet's birthday every 10 days.
def show_status(self)| Calls method to update the current status and displays it to the label.
def show_image(self, image_path)| method to display an image in a new window automatically destroying itself after a while.
def update_status(self, message)| Updates the current status of the pet.


In [9]:
class VirtualPetApp:
    '''
    class VirtualPetApp is used to visualize the VirtualPet and allows the user to change the pet scores by using
    different "buttons"
    '''

    def __init__(self, root):
        '''
        Initialize the VirtualPetApp
        '''
        self.root = root
        self.root.title("Virtual Pet")
        self.pet = None
        self.start_time = None  # Real start time
        self.real_seconds_elapsed = 0
        self.pet_seconds_elapsed = 0  # Pet time in seconds
        self.load_images()  # Load and scale images once
        self.create_widgets()
        self.load_pet_prompt()  # Check for saved pet data
        self.last_birthday_age = 0  # Track the last age when birthday was celebrated


    def load_images(self):  
        '''
    	load and scale images once during initialization
        '''
        self.animal_options = ["cat", "chicken", "shrimp", "sheep"]
        self.animal_images = {}
        for animal in self.animal_options:
            image_path = os.path.join('pet_pictures', f'picture_{animal}.png')  # Adjusted path to the images folder
            if not os.path.exists(image_path):
                continue
            image = Image.open(image_path)
            resized_image = image.resize((100, 100), Image.LANCZOS)  # Resize the image to a smaller format
            self.animal_images[animal] = ctk.CTkImage(light_image=resized_image, dark_image=resized_image,
                                                      size=(100, 100))  # Convert to CTkImage

        # Load button images
        self.play_image = ctk.CTkImage(
            light_image=Image.open('buttons/play_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/play_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))  
        self.feed_image = ctk.CTkImage(
            light_image=Image.open('buttons/feed_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/feed_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100)) 
        self.TV_image = ctk.CTkImage(light_image=Image.open('buttons/tv_button.png').resize((100, 100), Image.LANCZOS),
                                     dark_image=Image.open('buttons/tv_button.png').resize((100, 100), Image.LANCZOS),
                                     size=(100, 100))  
        self.sleep_image = ctk.CTkImage(
            light_image=Image.open('buttons/sleep_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/sleep_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))  
        self.vet_image = ctk.CTkImage(
            light_image=Image.open('buttons/vet_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/vet_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))

        # Load new button images
        self.start_image = ctk.CTkImage(
            light_image=Image.open('buttons/start_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/start_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))  
        self.load_image = ctk.CTkImage(
            light_image=Image.open('buttons/load_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/load_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))  
        self.save_image = ctk.CTkImage(
            light_image=Image.open('buttons/save_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/save_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100)) 
        self.check_status_image = ctk.CTkImage(
            light_image=Image.open('buttons/check_status_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/check_status_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))  
        self.quit_image = ctk.CTkImage(
            light_image=Image.open('buttons/quit_button.png').resize((100, 100), Image.LANCZOS),
            dark_image=Image.open('buttons/quit_button.png').resize((100, 100), Image.LANCZOS),
            size=(100, 100))  

    def create_widgets(self):
        '''
        create all buttons and widgets used on the 4 different frames.
        top_frame shows pet name
        middle_frame holds all buttons and can be changed by using the change_middle_frame_to functions
        bottom_frame shows a console
        time frame shows the current time/age of pet.
        '''
    
        # different frames    
        self.top_frame = ctk.CTkFrame(self.root)
        self.top_frame.pack(pady=10)

        self.middle_frame = ctk.CTkFrame(self.root)
        self.middle_frame.pack(pady=10)

        self.middle_frame_top = ctk.CTkFrame(self.middle_frame)  
        self.middle_frame_top.pack(pady=5)

        self.middle_frame_bottom = ctk.CTkFrame(self.middle_frame)  
        self.middle_frame_bottom.pack(pady=5)

        self.bottom_frame = ctk.CTkFrame(self.root)
        self.bottom_frame.pack(pady=10)

        self.time_frame = ctk.CTkFrame(self.root)
        self.time_frame.pack(pady=10)

        # Top Frame Widgets
        self.name_label = ctk.CTkLabel(self.top_frame, text="Enter pet's name:")
        self.name_label.pack(side="left", padx=5)

        self.name_entry = ctk.CTkEntry(self.top_frame)
        self.name_entry.pack(side="left", padx=5)

        # Middle Frame Widgets
        self.start_button = ctk.CTkButton(self.middle_frame_top, text="", image=self.start_image, compound="top",
                                          command=self.start)  
        self.start_button.pack(side="left", padx=5)

        self.load_button = ctk.CTkButton(self.middle_frame_top, text="", image=self.load_image, compound="top",
                                         command=self.load_pet_prompt)  
        self.load_button.pack(side="left", padx=5)

        self.save_button = ctk.CTkButton(self.middle_frame_top, text="", image=self.save_image, compound="top",
                                         command=self.save_pet,
                                         state="disabled")  
        self.save_button.pack(side="left", padx=5)

        self.feed_button = ctk.CTkButton(self.middle_frame_top, text="", image=self.feed_image, compound="top",
                                         command=self.show_feed_buttons, state="disabled")  
        self.feed_button.pack(side="left", padx=5)

        self.feed_pizza_button = ctk.CTkButton(self.middle_frame_bottom, text="Eat pizza",
                                               command=lambda: self.feed("pizza"))  
        self.feed_salad_button = ctk.CTkButton(self.middle_frame_bottom, text="Eat salad",
                                               command=lambda: self.feed("salad"))
        self.feed_barbecue_button = ctk.CTkButton(self.middle_frame_bottom, text="Have a barbecue",
                                                  command=lambda: self.feed("barbecue"))
        self.stop_eating_button = ctk.CTkButton(self.middle_frame_bottom, text="Finish eating",
                                                command=self.remove_feed_buttons)

        self.play_button = ctk.CTkButton(self.middle_frame_top, text="", image=self.play_image, compound="top",
                                         command=self.show_play_buttons,
                                         state="disabled")  
        self.play_button.pack(side="left", padx=5)

        self.play_hideandseek_button = ctk.CTkButton(self.middle_frame_bottom, text="Play hide and seek",
                                                     command=lambda: self.play(
                                                         "hide and seek"))  
        self.play_memory_button = ctk.CTkButton(self.middle_frame_bottom, text="Play memory", command=lambda: self.play(
            "memory")) 
        self.play_beachball_button = ctk.CTkButton(self.middle_frame_bottom, text="Play beachball",
                                                   command=lambda: self.play(
                                                       "beachball"))  
        self.stop_playing_button = ctk.CTkButton(self.middle_frame_bottom, text="Stop Playing",
                                                 command=self.remove_play_buttons)  

        self.TV_button = ctk.CTkButton(self.middle_frame_bottom, text="", image=self.TV_image, compound="top",
                                       command=self.TV,
                                       state="disabled")  
        self.TV_button.pack(side="left", padx=5)

        self.sleep_button = ctk.CTkButton(self.middle_frame_bottom, text="", image=self.sleep_image, compound="top",
                                          command=self.sleep,
                                          state="disabled")  
        self.sleep_button.pack(side="left", padx=5)

        self.vet_button = ctk.CTkButton(self.middle_frame_bottom, text="", image=self.vet_image, compound="top",
                                        command=self.vet,
                                        state="disabled")  
        self.vet_button.pack(side="left", padx=5)

        self.status_button = ctk.CTkButton(self.middle_frame_bottom, text="", image=self.check_status_image,
                                           compound="top", command=self.show_status,
                                           state="disabled")  
        self.status_button.pack(side="left", padx=5)

        self.quit_button = ctk.CTkButton(self.middle_frame_bottom, text="", image=self.quit_image, compound="top",
                                         command=self.root.destroy,
                                         state="normal")  
        self.quit_button.pack(side="left", padx=5)

        # Bottom Frame Widgets
        self.status_text = ctk.CTkTextbox(self.bottom_frame, height=140, width=550)
        self.status_text.pack(side="left", padx=5)

        # Time Frame Widgets
        self.real_time_label = ctk.CTkLabel(self.time_frame, text="Real Time: 0s", text_color="darkgreen")
        self.real_time_label.pack(side="left", padx=10)

        self.pet_time_label = ctk.CTkLabel(self.time_frame, text="Pet Time: 0 pet days", text_color="darkgreen")
        self.pet_time_label.pack(side="left", padx=10)

        self.pet_image_label = ctk.CTkLabel(self.root)
        self.pet_image_label.pack()  # To display the selected pet's image after naming

    def show_play_buttons(self):
        '''
        function to display play buttons to choose from different play activities.
        '''
        self.remove_feed_buttons() #
        self.play_hideandseek_button.pack(side="left", padx=5)  
        self.play_memory_button.pack(side="left", padx=5)  
        self.play_beachball_button.pack(side="left", padx=5)  
        self.stop_playing_button.pack(side="left", padx=5) 

    def remove_play_buttons(self):
        '''
        function called when "stop playing" button is clicked, removes the buttons for the mini games
        '''
        self.play_hideandseek_button.pack_forget()
        self.play_memory_button.pack_forget()
        self.play_beachball_button.pack_forget()
        self.stop_playing_button.pack_forget()

    def show_feed_buttons(self):  
        '''
        function to display food choice buttons
        '''
        self.remove_play_buttons()
        self.feed_pizza_button.pack(side="left", padx=5)
        self.feed_salad_button.pack(side="left", padx=5)
        self.feed_barbecue_button.pack(side="left", padx=5)
        self.stop_eating_button.pack(side="left", padx=5)

    def remove_feed_buttons(self):  
        '''
        function called when "stop eating" button is clicked, removes the buttons for the food choices
        '''
        self.feed_pizza_button.pack_forget()
        self.feed_salad_button.pack_forget()
        self.feed_barbecue_button.pack_forget()
        self.stop_eating_button.pack_forget()

    def start(self):
        '''
        Enable the buttons when the start button is pressed
        '''
        name = self.name_entry.get()
        if not name:
            messagebox.showwarning("Input Error", "Please enter a name for your pet.")
            return

        self.pet = VirtualPet(name)
        self.pet.selected_animal = self.selected_animal  #Save the selected animal
        self.start_time = time.time()  # Set the start time
        self.real_seconds_elapsed = 0  # Initialize real time elapsed
        self.pet_seconds_elapsed = 0  # Initialize pet time elapsed
        self.update_status(f"Welcome {self.pet.name}!")
        self.pet_image_label.configure(image=self.animal_images[self.selected_animal])  # Display selected pet's image
        self.save_button.configure(state=tk.NORMAL)
        self.feed_button.configure(state=tk.NORMAL)
        self.feed_pizza_button.configure(state=tk.NORMAL)
        self.feed_salad_button.configure(state=tk.NORMAL)
        self.feed_barbecue_button.configure(state=tk.NORMAL)
        self.play_button.configure(state=tk.NORMAL)
        self.TV_button.configure(state=tk.NORMAL)
        self.play_hideandseek_button.configure(state=tk.NORMAL)  
        self.play_memory_button.configure(state=tk.NORMAL)  
        self.play_beachball_button.configure(state=tk.NORMAL) 
        self.sleep_button.configure(state=tk.NORMAL)
        self.vet_button.configure(state=tk.NORMAL)
        self.status_button.configure(state=tk.NORMAL)
        self.quit_button.configure(state=tk.NORMAL)
        self.start_button.configure(state=tk.DISABLED)
        self.load_button.configure(state=tk.DISABLED)
        self.name_entry.configure(state=tk.DISABLED)

        self.update_times()
        self.update_pet_time()

    def pet_not_found(self):  
        '''
        handles pet not found scenario.  Adds the option to select a new pet if a pet is not found.
        '''
        messagebox.showinfo("Info", "Pet not found. Please select a new pet.")
        self.create_animal_selection_widgets()

    def create_animal_selection_widgets(self):
        '''
        choose an animal - allows user to choose a pet by clicking on the image-button with the favorite pet.
        '''
        self.animal_label = ctk.CTkLabel(self.root, text="Choose an animal:", text_color="darkgreen")
        self.animal_label.pack()

        self.animal_buttons = {}
        button_frame = ctk.CTkFrame(self.root)  # Create a frame to hold the buttons
        button_frame.pack()

        for animal in self.animal_options:
            self.animal_buttons[animal] = ctk.CTkButton(button_frame, image=self.animal_images[animal],
                                                        command=lambda a=animal: self.select_animal(a),
                                                        text="")  
            self.animal_buttons[animal].pack(side=tk.LEFT, padx=10, pady=10)  # Add padding for spacing

    def select_animal(self, animal):
        '''
        removes the buttons of unused animals and sets selected animal.
        '''
        self.selected_animal = animal

        for button in self.animal_buttons.values():
            button.pack_forget()
        self.animal_label.pack_forget()

        self.create_name_widgets()

    def create_name_widgets(self):
        '''
        packs name widgets on screen as well as the start button.
        '''
        self.name_label.pack()
        self.name_entry.pack()
        self.start_button.pack()

    def load_pet_prompt(self):
        '''
        used for loading prompt when game starts
        '''
        if os.path.exists(SAVE_FILE):
            with open(SAVE_FILE, "r") as file:  # Read mode
                pets_data = json.load(file)
            pet_names = list(pets_data.keys())
            if pet_names:
                pet_name = simpledialog.askstring("Load Pet", "Enter the name of the pet to load:",
                                                  initialvalue=pet_names[0])
                if pet_name and pet_name in pet_names:
                    self.load_pet(pet_name)
                else:
                    self.pet_not_found()  # Trigger the pet not found process
            else:
                messagebox.showinfo("Info", "No saved pets available.")
        else:
            self.pet_not_found()  # Trigger the pet not found process if save file doesn't exist

    def load_pet(self, pet_name): 
        '''
        Logic for loading old pet.
        '''
        try:
            with open(SAVE_FILE, "r") as file:  # reads saved file
                pets_data = json.load(file)
                if pet_name in pets_data:
                    pet_data = pets_data[pet_name]
                    self.pet = VirtualPet.from_dict(pet_data)
                    self.real_seconds_elapsed = pet_data['real_time_elapsed']
                    self.pet_seconds_elapsed = pet_data['pet_time_elapsed']
                    self.start_time = time.time() - self.real_seconds_elapsed
                    self.selected_animal = pet_data.get('selected_animal',
                                                        'cat')  # Default to 'cat' if not found
                    self.update_status(f"Welcome back {self.pet.name}!")  # Welcome message
                    self.pet_image_label.configure(
                        image=self.animal_images[self.selected_animal])  # Display the selected pet's image
                    self.save_button.configure(state=tk.NORMAL)
                    self.feed_button.configure(state=tk.NORMAL)
                    self.play_button.configure(state=tk.NORMAL)
                    self.feed_pizza_button.configure(state=tk.NORMAL)
                    self.feed_salad_button.configure(state=tk.NORMAL)
                    self.feed_barbecue_button.configure(state=tk.NORMAL)
                    self.TV_button.configure(state=tk.NORMAL)
                    self.play_hideandseek_button.configure(state=tk.NORMAL)  
                    self.play_memory_button.configure(state=tk.NORMAL)  
                    self.play_beachball_button.configure(state=tk.NORMAL)  
                    self.sleep_button.configure(state=tk.NORMAL)
                    self.vet_button.configure(state=tk.NORMAL)
                    self.status_button.configure(state=tk.NORMAL)
                    self.quit_button.configure(state=tk.NORMAL)
                    self.start_button.configure(state=tk.DISABLED)
                    self.load_button.configure(state=tk.DISABLED)
                    self.name_entry.configure(state=tk.DISABLED)

                    self.update_times()
                    self.update_pet_time()
                else:
                    messagebox.showerror("Error", "Pet not found in the save file.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load pet: {e}")


    def save_pet(self):
        '''
        Save current playing pet
        '''
        if not self.pet:
            return
        try:
            if os.path.exists(SAVE_FILE):
                with open(SAVE_FILE, "r") as file:
                    pets_data = json.load(file)
            else:
                pets_data = {}

            pets_data[self.pet.name] = self.pet.to_dict(self.real_seconds_elapsed, self.pet_seconds_elapsed)

            with open(SAVE_FILE, "w") as file:  # Sets Write mode
                json.dump(pets_data, file, indent=4)

            self.update_status(f"{self.pet.name}'s data has been saved.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save pet: {e}")


    def update_times(self):
        '''
        Function that sets how the actual time passes.
        '''
        if self.pet:
            # Real time elapsed since start
            current_time = time.time()
            real_time_elapsed = current_time - self.start_time
            self.real_seconds_elapsed = int(real_time_elapsed)
            self.real_time_label.configure(text=f"Real Time: {self.format_real_time(self.real_seconds_elapsed)}")

        # Schedule the update_times method to run again after 1000ms (1s)
        self.root.after(1000, self.update_times)


    def update_pet_time(self):
        '''
        Function that moderates the pet time (artificial one). We can edit here to choose how the pet time passes.
        '''
        if self.pet:
            # Update pet time independently
            self.pet_seconds_elapsed += 3600 / 24  # Increment pet time by 1 pet hour (3600 pet seconds)
            pet_days_elapsed = self.pet_seconds_elapsed / (24 * 60 * 60)
            old_age = self.pet.age
            self.pet.age = int(pet_days_elapsed)  # Update pet age in pet days
            if self.pet.age > old_age:
                self.update_status(f"A new pet day has passed. {self.pet.name} is now {self.pet.age} pet days old.")
                if self.pet.age % 10 == 0 and self.pet.age > self.last_birthday_age:
                    self.celebrate_birthday()
            self.pet_time_label.configure(text=f"Pet Time: {self.format_pet_time(self.pet_seconds_elapsed)}")

        # Schedule the update_pet_time method to run again after 1s
        self.root.after(1000, self.update_pet_time)

    def format_real_time(self, seconds):
        '''
        Function to set the format for the actual time passed.
        '''
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)
        days, hours = divmod(hours, 24)
        return f"{days}d {hours}h {minutes}m {seconds}s"

    
    def format_pet_time(self, pet_seconds):
        '''
        Formats the Pet time (Artificial one)
        '''
        pet_hours, pet_seconds = divmod(pet_seconds, 3600)
        pet_days, pet_hours = divmod(pet_hours, 24)
        pet_minutes, pet_seconds = divmod(pet_seconds, 60)
        return f"{int(pet_days)} pet days {int(pet_hours)}h {int(pet_minutes)}m {int(pet_seconds)}s"

    
    def feed(self, food_type):
        '''
        action caused by clicking feed-button. Calls feed function of pet.
        '''
        if not self.pet:
            return
        result = self.pet.feed(food_type)
        self.update_status(result)
        self.advance_pet_time(2 * 3600)  # Advance pet time by 2 pet hours (converted to seconds)
        self.show_image(f"feed_{self.pet.selected_animal}.png")  # Display image after feeding based on selected pet

    
    def play(self, game_type):
        '''
        Action caused by clicking play-button. Calls play function of pet.
        '''
        if not self.pet:
            return
        result = self.pet.play(game_type)
        self.update_status(result)
        self.advance_pet_time(2 * 3600)  # Advance pet time by 2 pet hours (converted to seconds)
        self.show_image(f"play_{self.pet.selected_animal}.png")  #dissplay image after playing based on selected pet

    
    def TV(self):
        '''
        Action causes by clicking TV-button.
        '''
        if not self.pet:
            return
        result = self.pet.TV()
        self.update_status(result)
        self.advance_pet_time(2 * 3600)  # Advance pet time by 2 pet hours (converted to seconds)
        self.show_image(f"tv_{self.pet.selected_animal}.png")  # display image after watching TV based on selected pet


    def sleep(self):
        '''
        Action caused by clicking sleep button. Calls sleep function of VirtualPet.
        '''
        if not self.pet:
            return
        result = self.pet.sleep()
        self.update_status(result)
        self.advance_pet_time(2 * 3600)  # Advance pet time by 2 pet hours (converted to seconds)
        self.show_image(f"sleep_{self.pet.selected_animal}.png")  # display image after sleeping based on selected pet


    def vet(self):
        '''
        Action caused by vet button. Calls vet function of VirtualPet.
        '''
        if not self.pet:
            return
        result = self.pet.vet()
        self.update_status(result)
        self.advance_pet_time(2 * 3600)  # Advance pet time by 2 pet hours (converted to seconds)
        self.show_image(f"vet_{self.pet.selected_animal}.png")  # display image after visiting the vet based on selected pet

    def advance_pet_time(self, seconds):
        '''
        This snippet provides an output in the console on each new pet day and checks for birthdays.
        '''
        initial_days_elapsed = self.pet_seconds_elapsed / (24 * 60 * 60)
        self.pet_seconds_elapsed += seconds  # Advance pet time by given seconds
        final_days_elapsed = self.pet_seconds_elapsed / (24 * 60 * 60)

        initial_age = int(initial_days_elapsed)
        final_age = int(final_days_elapsed)

        if final_age > initial_age:
            for age in range(initial_age + 1, final_age + 1):
                self.update_status(f"A new pet day has passed. {self.pet.name} is now {age} pet days old.")
            if age % 10 == 0 and age > self.last_birthday_age:
                self.celebrate_birthday()

        self.pet.age = final_age  # Update pet age in pet days
        self.pet_time_label.configure(text=f"Pet Time: {self.format_pet_time(self.pet_seconds_elapsed)}")

        
    def celebrate_birthday(self):
        '''
        Function to handle the pet's birthday.
        '''
        self.update_status(f"It's {self.pet.name}'s birthday!")
        birthday_image_path = ("happy_birthday.png")
        self.show_image(birthday_image_path)


    def show_status(self):
        '''
        calls function to update the current status and displays it to the console.
        '''
        if not self.pet:
            return
        status = self.pet.show_status()
        self.update_status(status)


    def show_image(self, image_path):
        '''
        Display an image in a new window.
        '''
        window = tk.Toplevel(self.root)
        img = ImageTk.PhotoImage(Image.open(image_path))
        panel = tk.Label(window, image=img)
        panel.image = img  # Keep a reference to avoid garbage collection
        panel.pack()

        #option 1:
        #window.after(8000, window.destroy)  # Close the window after 5 seconds

        #option 2:
        save_button = ctk.CTkButton(window, text="ok", command= window.destroy) # alternatively we could close the window with a button
        save_button.pack(side = "bottom")


    def update_status(self, message):
        self.status_text.insert(tk.END, f"{message}\n")
        self.status_text.see(tk.END)


Finally, to get the programme started, we implemented the following logic, initializing an instance of the VirtualPetApp with a toplevel Tkinter widget as argument and the mainloop function call to keep it running until the user closes the game.

In [10]:
if __name__ == "__main__":
    root = tk.Tk()
    app = VirtualPetApp(root)
    root.mainloop()